# Analyse des GTFS (trouver meilleur titre)

Ce notebook sert à explorer les données GTFS blablabla...

In [6]:
import os
import sys
from pathlib import Path
import pandas as pd
import gtfs_kit as gk
import folium

sys.path.append('..')

from main import (
    charger_gtfs,
    obtenir_service_ids_pour_date,
    afficher_statistiques,
    exporter_resultats,
    carte_arrets,
)

In [7]:
# Saisir la date pour laquelle on veut récupérer les données :
DATE_ANALYSE = "20251125"  # Format AAAAMMJJ

In [8]:
# Charger les GTFS à analayser
GTFS_ZIP_PATH = "../data/TAM_MMM_GTFS.zip" # à modifier selon le GTFS à analyser

feed = charger_gtfs(GTFS_ZIP_PATH)

Chargement du fichier GTFS : ../data/TAM_MMM_GTFS.zip
✓ GTFS chargé avec succès


La cellule suivante défini la routine de création des indicateurs. \
Vous pouvez la déplier et la modifier en fonction de vos besoins.

In [9]:
def calculer_indicateurs_arrets(feed, date_str):
    """
    Calcule les indicateurs pour chaque arrêt :
    - Nombre de lignes desservies
    - Nombre de passages
    - Heure du premier départ
    - Heure du dernier départ
    - Amplitude horaire
    - Temps d'attente moyen, min et max
    """
    print(f"\nCalcul des indicateurs pour le {date_str}...")

    # Récupérer les services actifs
    service_ids = obtenir_service_ids_pour_date(feed, date_str)

    if not service_ids:
        print("⚠ Aucun service actif pour cette date")
        return None

    # Filtrer les trips actifs ce jour-là
    trips_actifs = feed.trips[feed.trips["service_id"].isin(service_ids)]
    print(f"✓ {len(trips_actifs)} trips actifs")

    # Joindre avec stop_times
    stop_times_actifs = feed.stop_times.merge(
        trips_actifs[["trip_id", "service_id"]], on="trip_id"
    )

    # Calculer les indicateurs par arrêt
    indicateurs = (
        stop_times_actifs.groupby("stop_id")
        .agg(
            nombre_passages=("trip_id", "count"),
            premier_depart=("departure_time", "min"),
            dernier_depart=("departure_time", "max"),
        )
        .reset_index()
    )


    # Utilisation de gtfs_kit pour calculer les stats de headway
    indic_gk = feed.compute_stop_stats([date_str])
    indicateurs = indicateurs.merge(
        indic_gk[["stop_id", "mean_headway", "max_headway", "num_routes"]],
        on="stop_id",
        how="left",
    )
    indicateurs.rename(columns={
        "num_routes": "nb_lignes",
        "mean_headway": "temps_attente_moyen",
        "max_headway": "temps_attente_max"
    }, inplace=True)
 
    # Joindre avec les informations des arrêts
    indicateurs = indicateurs.merge(
        feed.stops[["stop_id", "stop_name", "stop_lat", "stop_lon"]],
        on="stop_id",
        how="left",
    )

    # Calculer l'amplitude horaire
    indicateurs["amplitude_horaire"] = pd.to_timedelta(
        indicateurs["dernier_depart"]
    ) - pd.to_timedelta(indicateurs["premier_depart"])
    # nettoyer pour retirer le 0 days
    indicateurs["amplitude_horaire"] = indicateurs["amplitude_horaire"].apply(
        lambda x: str(x).replace("0 days ", " ").strip()
    )

    # Réorganiser les colonnes
    indicateurs = indicateurs[
        [
            "stop_id",
            "stop_name",
            "stop_lat",
            "stop_lon",
            "nb_lignes",
            "nombre_passages",
            "premier_depart",
            "dernier_depart",
            "amplitude_horaire",
            "temps_attente_moyen",
            "temps_attente_max",
        ]
    ]

    # Trier par nombre de passages décroissant
    indicateurs = indicateurs.sort_values("nombre_passages", ascending=False)

    print(f"✓ Indicateurs calculés pour {len(indicateurs)} arrêts")

    return indicateurs

indicateurs = calculer_indicateurs_arrets(feed, DATE_ANALYSE)

indicateurs.drop(columns=["stop_lat", "stop_lon"]).head()


Calcul des indicateurs pour le 20251125...
✓ Services actifs le 20251125 : 4 service(s)
✓ 4165 trips actifs
✓ Indicateurs calculés pour 1401 arrêts


,stop_id,stop_name,nb_lignes,nombre_passages,premier_depart,dernier_depart,amplitude_horaire,temps_attente_moyen,temps_attente_max
123,1130,Mosson,1.0,406,04:27:00,25:48:00,21:21:00,2.372093,7.0
141,1147,Gare Saint-Roch,2.0,362,05:05:00,25:30:00,20:25:00,2.633700,6.0
167,1170,Gare Saint-Roch,2.0,362,05:04:00,25:15:00,20:11:00,2.616788,6.0
168,1171,Comédie,2.0,362,05:06:00,25:17:00,20:11:00,2.614545,6.0
140,1146,Comédie,2.0,362,05:03:00,25:28:00,20:25:00,2.637363,6.0


In [28]:
afficher_statistiques(indicateurs)

exporter_resultats(indicateurs, DATE_ANALYSE)


STATISTIQUES GLOBALES
Nombre total d'arrêts desservis : 1401
Nombre total de passages : 89,477
Moyenne de passages par arrêt : 63.9
Médiane de passages par arrêt : 41.0

Arrêt le plus fréquenté : Mosson (406 passages)
Premier départ global : 04:16:00
Dernier départ global : 25:51:00

TOP 10 DES ARRÊTS LES PLUS FRÉQUENTÉS
stop_id                 stop_name  stop_lat  stop_lon  nombre_passages premier_depart dernier_depart amplitude_horaire  headway_moyen  headway_max
   1130                    Mosson 43.616320  3.819787              406       04:27:00       25:48:00          21:21:00       3.162963         34.0
   1147           Gare Saint-Roch 43.605739  3.880156              362       05:05:00       25:30:00          20:25:00       3.393352         30.0
   1170           Gare Saint-Roch 43.605766  3.880181              362       05:04:00       25:15:00          20:11:00       3.354571         15.0
   1171                   Comédie 43.608361  3.879732              362       05:06:00   

In [11]:
# Première carte : arrêts et passages

carte1 = carte_arrets(indicateurs)

carte1


✓ Carte des arrêts enregistrée dans : stops_map.html
Type of m: <class 'folium.folium.Map'>
